<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План
    1. Введение
    2. Описание данных
    3. Предобработка данных
    4. Обучение и проверка моделей
    5. Заключение

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l stackoverflow.10kk.tsv

10000000 stackoverflow.10kk.tsv
CPU times: user 1.08 s, sys: 280 ms, total: 1.36 s
Wall time: 34.8 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [6]:
import os
import re
import io
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score


DATA_DIR = '../../../data'
taglist = ['javascript', 'java', 'python', 'ruby', 'php', 'c++', 'c#', 'go', 'scala', 'swift']

def to_vw_format(document, label=None):
    return str(label or '') + ' | ' + re.sub('[:|]', '', document.lower()) + '\n'

def preprocess(tsv_file, vw_file, tags=taglist):
    tags_set = set(tags)
    
    with io.open(tsv_file, 'r', encoding='utf-8') as source:
        with io.open(vw_file, 'w', encoding='utf-8') as target:
            for i, line in enumerate(tqdm(source)):
                line = line.strip()
                q_tag = line.split('\t')

                if (len(q_tag) != 2):
                    continue
                
                text = q_tag[0]
                labels = q_tag[1].split(' ')
                intersection = tags_set.intersection(labels)
                
                if len(intersection) != 1:
                    continue
                
                target.write(to_vw_format(text, tags.index(intersection.pop()) + 1))
        

In [46]:

preprocess(os.path.join(DATA_DIR, 'stackoverflow.10kk.tsv'), os.path.join(DATA_DIR, 'stackoverflow.10kk.processed.vw'))


0it [00:00, ?it/s]
2797it [00:00, 27968.43it/s]
8570it [00:00, 42847.59it/s]
15609it [00:00, 50348.76it/s]
22337it [00:00, 54610.58it/s]
29416it [00:00, 57788.44it/s]
37162it [00:00, 61017.86it/s]
44699it [00:00, 62952.74it/s]
51691it [00:00, 63891.30it/s]
59195it [00:00, 65117.29it/s]
66088it [00:01, 65236.17it/s]
73437it [00:01, 66036.70it/s]
80425it [00:01, 65918.37it/s]
87962it [00:01, 66684.59it/s]
95698it [00:01, 67436.60it/s]
102958it [00:01, 63394.17it/s]
110195it [00:01, 63951.66it/s]
117204it [00:01, 64288.16it/s]
124811it [00:01, 64900.61it/s]
132163it [00:02, 65326.48it/s]
139218it [00:02, 65479.79it/s]
146628it [00:02, 65866.86it/s]
153962it [00:02, 66187.96it/s]
161536it [00:02, 66554.08it/s]
168836it [00:02, 66756.17it/s]
176278it [00:02, 67047.52it/s]
183730it [00:02, 67321.18it/s]
191074it [00:02, 67513.46it/s]
198418it [00:02, 67600.22it/s]
205657it [00:03, 67757.90it/s]
213240it [00:03, 68015.25it/s]
220879it [00:03, 68295.11it/s]
228495it [00:03, 68530.88it/s]
2359

3870192it [01:08, 56677.95it/s]
3877768it [01:08, 56706.68it/s]
3885251it [01:08, 56733.14it/s]
3892797it [01:08, 56759.61it/s]
3900318it [01:08, 56786.47it/s]
3907814it [01:08, 56813.71it/s]
3915450it [01:08, 56842.09it/s]
3922855it [01:08, 56861.26it/s]
3930115it [01:09, 56880.74it/s]
3937312it [01:09, 56899.25it/s]
3944426it [01:09, 56912.41it/s]
3951426it [01:09, 56932.08it/s]
3958382it [01:09, 56946.14it/s]
3965239it [01:10, 56522.72it/s]
3972650it [01:10, 56549.36it/s]
3980315it [01:10, 56577.12it/s]
3988000it [01:10, 56606.70it/s]
3995479it [01:10, 56632.46it/s]
4002882it [01:10, 56657.08it/s]
4010477it [01:10, 56684.35it/s]
4018068it [01:10, 56711.48it/s]
4025621it [01:10, 56738.00it/s]
4033149it [01:11, 56763.29it/s]
4040597it [01:11, 56785.79it/s]
4048001it [01:11, 56808.41it/s]
4055362it [01:11, 56832.75it/s]
4062723it [01:11, 56850.65it/s]
4069939it [01:12, 56445.30it/s]
4077525it [01:12, 56472.97it/s]
4084927it [01:12, 56497.23it/s]
4092479it [01:12, 56523.50it/s]
4099929i

7734473it [02:19, 55458.21it/s]
7742062it [02:20, 55194.02it/s]
7749682it [02:20, 55209.38it/s]
7757155it [02:20, 55222.88it/s]
7764455it [02:20, 55235.92it/s]
7771695it [02:20, 55248.12it/s]
7779026it [02:20, 55260.55it/s]
7786560it [02:20, 55275.20it/s]
7794040it [02:20, 55289.04it/s]
7801634it [02:21, 55303.68it/s]
7809340it [02:21, 55319.09it/s]
7817027it [02:21, 55334.34it/s]
7824565it [02:21, 55348.13it/s]
7832103it [02:21, 55361.11it/s]
7839729it [02:21, 55376.26it/s]
7847310it [02:21, 55390.68it/s]
7854847it [02:22, 55194.92it/s]
7862125it [02:22, 55207.26it/s]
7870126it [02:22, 55224.67it/s]
7877772it [02:22, 55239.94it/s]
7885272it [02:22, 55253.78it/s]
7892882it [02:22, 55268.38it/s]
7900617it [02:22, 55283.83it/s]
7908273it [02:23, 55298.70it/s]
7915909it [02:23, 55313.42it/s]
7923616it [02:23, 55328.22it/s]
7931232it [02:23, 55343.14it/s]
7938995it [02:23, 55358.68it/s]
7946639it [02:23, 55371.05it/s]
7954174it [02:24, 55163.32it/s]
7961406it [02:24, 55175.59it/s]
7969134i

Должно получиться вот такое число строк – 4389054. 10 Гб у меня обработались примерно за 2 минуты.

In [4]:
# !python3.5 preprocess.py stackoverflow.10kk.tsv stackoverflow.vw

10000000it [02:19, 71802.03it/s]
4389054 lines selected, 15 lines corrupted.


In [7]:
%%time
!wc -l ../../../data/stackoverflow.10kk.processed.vw

4389054 ../../../data/stackoverflow.10kk.processed.vw
Wall time: 10.1 s


In [8]:
!gzip ../../../data/stackoverflow.10kk.tsv

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [5]:
# !head -n 1463018 ../../../data/stackoverflow.10kk.processed.vw > ../../../data/stackoverflow_train.vw

### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: `loss_function=hinge`, `bit_precision`=28 и `seed`=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [16]:
from subprocess import call

valid_labels = []
with io.open('../../../data/stackoverflow_valid_labels.txt', encoding='utf-8') as vl:
    valid_labels = [float(label) for label in vl.readlines()]
    
accuracies = {}
for ps in [1, 3, 5]:
    for ng in [1, 2, 3]:
        model_name = 'stackoverflow_model_p{}_ng{}.vw'.format(ps, ng)
        rc = call(["vw", "--oaa", '10', '-d', '../../../data/stackoverflow_train.vw', 
              '--loss_function', 'hinge', '-b', '28', '--random_seed', '17',
             '-f', model_name, '--passes', str(ps), '--ngram', str(ng), '-c' ])
        if rc == 0:
            pred_name = 'stackoverflow_valid_predictions_p{}_ng{}.txt'.format(ps, ng)
            call(['vw', '-i', model_name, '-t', '-d', '../../../data/stackoverflow_valid.vw', '-p', pred_name])
            
            with io.open(pred_name, encoding='utf-8') as pred_file:
                valid_prediction = [float(label) for label in pred_file.readlines()]
                accuracies[model_name] = accuracy_score(valid_labels, valid_prediction)
        else:
            print('Error while fitting the model' + model_name)

In [22]:
import operator

sorted_x = sorted(accuracies.items(), key=operator.itemgetter(1))

In [26]:
sorted_x

[('stackoverflow_model_p1_ng1.vw', 0.9166626794748937),
 ('stackoverflow_model_p3_ng1.vw', 0.9180078440593349),
 ('stackoverflow_model_p5_ng1.vw', 0.9180269825798453),
 ('stackoverflow_model_p3_ng3.vw', 0.9272544835401888),
 ('stackoverflow_model_p5_ng3.vw', 0.9281464752996887),
 ('stackoverflow_model_p1_ng3.vw', 0.9299701028968885),
 ('stackoverflow_model_p3_ng2.vw', 0.9305374233263022),
 ('stackoverflow_model_p5_ng2.vw', 0.93123529580634),
 ('stackoverflow_model_p1_ng2.vw', 0.932356949811964)]

<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [27]:
pred_test_name = 'stackoverflow_test_predictions_p1_ng2.txt'
call(['vw', '-i', sorted_x[-1][0], '-t', '-d', '../../../data/stackoverflow_test.vw', '-p', pred_test_name])


test_labels = []
with io.open('../../../data/stackoverflow_test_labels.txt', encoding='utf-8') as tl:
    test_labels = [float(label) for label in tl.readlines()]

test_acc = 0
with io.open(pred_test_name, encoding='utf-8') as pred_file:
    test_prediction = [float(label) for label in pred_file.readlines()]
    test_acc = accuracy_score(test_labels, test_prediction)

In [30]:
(test_acc - sorted_x[-1][1]) * 100

0.017771483331030513

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [31]:
!cat ../../../data/stackoverflow_train.vw ../../../data/stackoverflow_valid.vw > ../../../data/stackoverflow_train_valid.vw  

In [32]:
model_name = 'stackoverflow_model_p1_ng2_tr_v.vw'
tr_val_test_acc = 0

rc = call(["vw", "--oaa", '10', '-d', '../../../data/stackoverflow_train_valid.vw', 
      '--loss_function', 'hinge', '-b', '28', '--random_seed', '17',
     '-f', model_name, '--passes', '1', '--ngram', '2', '-c' ])
if rc == 0:
    pred_name = 'stackoverflow_test_predictions_p1_ng2_tr_v.txt'
    call(['vw', '-i', model_name, '-t', '-d', '../../../data/stackoverflow_test.vw', '-p', pred_name])

    with io.open(pred_name, encoding='utf-8') as pred_file:
        test_prediction = [float(label) for label in pred_file.readlines()]
        tr_val_test_acc = accuracy_score(test_labels, test_prediction)

In [33]:
(tr_val_test_acc - sorted_x[-1][1]) * 100

0.4090175240496041

In [34]:
(tr_val_test_acc - test_acc) * 100

0.3912460407185736

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)